<a href="https://colab.research.google.com/github/Jirtus-sanasam/MLP-Diabetes/blob/main/Diabetes7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Initial import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("/content/diabetes_data2.csv")

In [5]:
# Separate the independent and dependent variable

X = df.drop("Outcome", axis=1)
Y = df['Outcome']

In [6]:
# Split full dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=Y  # important for imbalanced medical data
)


In [7]:
# Split training data into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train,
    test_size=0.2,
    random_state=42,
    stratify=Y_train
)


In [8]:
# Initialize scaler
scaler = StandardScaler()

# Fit only on training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform validation and test data
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [9]:
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)


In [10]:
# Data for GA hyperparameter tuning
X_ga_train = X_train_scaled
Y_ga_train = Y_train

X_ga_val = X_val_scaled
Y_ga_val = Y_val


In [11]:
"""
Diabetes Prediction using MLP with Nested Genetic Algorithm Optimization
=========================================================================

Workflow:
1. Outer GA (Meta-GA): Optimizes GA hyperparameters
2. Inner GA: Optimizes MLP hyperparameters using parameters from Outer GA
3. MLP: Trains and predicts diabetes using hyperparameters from Inner GA
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')


class MLPGeneticAlgorithm:
    """Inner GA: Optimizes MLP hyperparameters"""

    def __init__(self, X_train, y_train, X_val, y_val,
                 population_size=20, generations=15,
                 mutation_rate=0.1, crossover_rate=0.8):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

        # MLP hyperparameter search space
        self.param_ranges = {
            'hidden_layer_1': (10, 200),      # neurons in first hidden layer
            'hidden_layer_2': (5, 150),       # neurons in second hidden layer
            'learning_rate_init': (0.0001, 0.1),  # learning rate
            'alpha': (0.0001, 0.01),          # L2 regularization
            'batch_size': (16, 128)           # batch size
        }

        self.best_individual = None
        self.best_fitness = -np.inf
        self.fitness_history = []

    def create_individual(self):
        """Create a random MLP configuration"""
        individual = {
            'hidden_layer_1': np.random.randint(
                self.param_ranges['hidden_layer_1'][0],
                self.param_ranges['hidden_layer_1'][1]
            ),
            'hidden_layer_2': np.random.randint(
                self.param_ranges['hidden_layer_2'][0],
                self.param_ranges['hidden_layer_2'][1]
            ),
            'learning_rate_init': np.random.uniform(
                self.param_ranges['learning_rate_init'][0],
                self.param_ranges['learning_rate_init'][1]
            ),
            'alpha': np.random.uniform(
                self.param_ranges['alpha'][0],
                self.param_ranges['alpha'][1]
            ),
            'batch_size': np.random.randint(
                self.param_ranges['batch_size'][0],
                self.param_ranges['batch_size'][1]
            )
        }
        return individual

    def evaluate_fitness(self, individual):
        """Evaluate MLP with given hyperparameters"""
        try:
            mlp = MLPClassifier(
                hidden_layer_sizes=(
                    individual['hidden_layer_1'],
                    individual['hidden_layer_2']
                ),
                learning_rate_init=individual['learning_rate_init'],
                alpha=individual['alpha'],
                batch_size=individual['batch_size'],
                max_iter=200,
                random_state=42,
                early_stopping=True,
                validation_fraction=0.1
            )

            mlp.fit(self.X_train, self.y_train)
            y_pred = mlp.predict(self.X_val)

            # Fitness is a combination of accuracy and F1 score
            accuracy = accuracy_score(self.y_val, y_pred)
            f1 = f1_score(self.y_val, y_pred, average='weighted')
            fitness = 0.6 * accuracy + 0.4 * f1

            return fitness
        except Exception as e:
            return 0.0  # Return low fitness if model fails

    def selection(self, population, fitness_scores):
        """Tournament selection"""
        tournament_size = 3
        selected = []

        for _ in range(len(population)):
            tournament_idx = np.random.choice(
                len(population), tournament_size, replace=False
            )
            tournament_fitness = [fitness_scores[i] for i in tournament_idx]
            winner_idx = tournament_idx[np.argmax(tournament_fitness)]
            selected.append(population[winner_idx].copy())

        return selected

    def crossover(self, parent1, parent2):
        """Single-point crossover"""
        if np.random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()

        child1, child2 = parent1.copy(), parent2.copy()

        # Randomly choose which parameters to swap
        for key in parent1.keys():
            if np.random.random() < 0.5:
                child1[key], child2[key] = child2[key], child1[key]

        return child1, child2

    def mutate(self, individual):
        """Gaussian mutation for continuous, random reset for discrete"""
        mutated = individual.copy()

        for key, value in mutated.items():
            if np.random.random() < self.mutation_rate:
                if key in ['hidden_layer_1', 'hidden_layer_2', 'batch_size']:
                    # Discrete parameters
                    mutated[key] = np.random.randint(
                        self.param_ranges[key][0],
                        self.param_ranges[key][1]
                    )
                else:
                    # Continuous parameters - Gaussian mutation
                    mutation = np.random.normal(0, 0.1 * value)
                    mutated[key] = np.clip(
                        value + mutation,
                        self.param_ranges[key][0],
                        self.param_ranges[key][1]
                    )

        return mutated

    def evolve(self):
        """Run the genetic algorithm"""
        # Initialize population
        population = [self.create_individual() for _ in range(self.population_size)]

        print(f"\n{'='*60}")
        print(f"Inner GA: Optimizing MLP Hyperparameters")
        print(f"Population Size: {self.population_size}, Generations: {self.generations}")
        print(f"Mutation Rate: {self.mutation_rate:.3f}, Crossover Rate: {self.crossover_rate:.3f}")
        print(f"{'='*60}")

        for generation in range(self.generations):
            # Evaluate fitness
            fitness_scores = [self.evaluate_fitness(ind) for ind in population]

            # Track best individual
            max_fitness_idx = np.argmax(fitness_scores)
            if fitness_scores[max_fitness_idx] > self.best_fitness:
                self.best_fitness = fitness_scores[max_fitness_idx]
                self.best_individual = population[max_fitness_idx].copy()

            avg_fitness = np.mean(fitness_scores)
            self.fitness_history.append((generation, avg_fitness, self.best_fitness))

            print(f"Gen {generation+1:2d} | Avg Fitness: {avg_fitness:.4f} | "
                  f"Best Fitness: {self.best_fitness:.4f}")

            # Selection
            selected = self.selection(population, fitness_scores)

            # Crossover and mutation
            next_population = []
            for i in range(0, len(selected), 2):
                parent1 = selected[i]
                parent2 = selected[i+1] if i+1 < len(selected) else selected[0]

                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                next_population.extend([child1, child2])

            # Elitism: keep best individual
            next_population[0] = self.best_individual.copy()
            population = next_population[:self.population_size]

        print(f"\nBest MLP Configuration Found:")
        for key, value in self.best_individual.items():
            print(f"  {key}: {value}")

        return self.best_individual, self.best_fitness


class MetaGeneticAlgorithm:
    """Outer GA: Optimizes GA hyperparameters"""

    def __init__(self, X_train, y_train, X_val, y_val,
                 population_size=10, generations=5):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.population_size = population_size
        self.generations = generations

        # GA hyperparameter search space
        self.param_ranges = {
            'population_size': (10, 40),
            'generations': (10, 30),
            'mutation_rate': (0.05, 0.3),
            'crossover_rate': (0.6, 0.95)
        }

        self.best_individual = None
        self.best_fitness = -np.inf
        self.fitness_history = []

    def create_individual(self):
        """Create random GA configuration"""
        individual = {
            'population_size': np.random.randint(
                self.param_ranges['population_size'][0],
                self.param_ranges['population_size'][1]
            ),
            'generations': np.random.randint(
                self.param_ranges['generations'][0],
                self.param_ranges['generations'][1]
            ),
            'mutation_rate': np.random.uniform(
                self.param_ranges['mutation_rate'][0],
                self.param_ranges['mutation_rate'][1]
            ),
            'crossover_rate': np.random.uniform(
                self.param_ranges['crossover_rate'][0],
                self.param_ranges['crossover_rate'][1]
            )
        }
        return individual

    def evaluate_fitness(self, individual):
        """Run Inner GA with these GA parameters and return best MLP performance"""
        inner_ga = MLPGeneticAlgorithm(
            self.X_train, self.y_train, self.X_val, self.y_val,
            population_size=individual['population_size'],
            generations=individual['generations'],
            mutation_rate=individual['mutation_rate'],
            crossover_rate=individual['crossover_rate']
        )

        _, fitness = inner_ga.evolve()
        return fitness

    def selection(self, population, fitness_scores):
        """Tournament selection"""
        tournament_size = 2
        selected = []

        for _ in range(len(population)):
            tournament_idx = np.random.choice(
                len(population), tournament_size, replace=False
            )
            tournament_fitness = [fitness_scores[i] for i in tournament_idx]
            winner_idx = tournament_idx[np.argmax(tournament_fitness)]
            selected.append(population[winner_idx].copy())

        return selected

    def crossover(self, parent1, parent2):
        """Uniform crossover"""
        child1, child2 = parent1.copy(), parent2.copy()

        for key in parent1.keys():
            if np.random.random() < 0.5:
                child1[key], child2[key] = child2[key], child1[key]

        return child1, child2

    def mutate(self, individual):
        """Gaussian mutation"""
        mutated = individual.copy()
        mutation_prob = 0.3

        for key, value in mutated.items():
            if np.random.random() < mutation_prob:
                if key in ['population_size', 'generations']:
                    mutated[key] = np.random.randint(
                        self.param_ranges[key][0],
                        self.param_ranges[key][1]
                    )
                else:
                    mutation = np.random.normal(0, 0.1 * value)
                    mutated[key] = np.clip(
                        value + mutation,
                        self.param_ranges[key][0],
                        self.param_ranges[key][1]
                    )

        return mutated

    def evolve(self):
        """Run the meta genetic algorithm"""
        population = [self.create_individual() for _ in range(self.population_size)]

        print(f"\n{'#'*60}")
        print(f"OUTER GA (Meta-GA): Optimizing GA Hyperparameters")
        print(f"Population Size: {self.population_size}, Generations: {self.generations}")
        print(f"{'#'*60}\n")

        for generation in range(self.generations):
            print(f"\n{'*'*60}")
            print(f"META-GA Generation {generation+1}/{self.generations}")
            print(f"{'*'*60}")

            # Evaluate fitness
            fitness_scores = []
            for idx, ind in enumerate(population):
                print(f"\nEvaluating GA Config {idx+1}/{len(population)}:")
                print(f"  Population: {ind['population_size']}, Generations: {ind['generations']}")
                print(f"  Mutation: {ind['mutation_rate']:.3f}, Crossover: {ind['crossover_rate']:.3f}")

                fitness = self.evaluate_fitness(ind)
                fitness_scores.append(fitness)

                print(f"  => Fitness: {fitness:.4f}")

            # Track best
            max_fitness_idx = np.argmax(fitness_scores)
            if fitness_scores[max_fitness_idx] > self.best_fitness:
                self.best_fitness = fitness_scores[max_fitness_idx]
                self.best_individual = population[max_fitness_idx].copy()

            avg_fitness = np.mean(fitness_scores)
            self.fitness_history.append((generation, avg_fitness, self.best_fitness))

            print(f"\n{'*'*60}")
            print(f"Meta-GA Gen {generation+1} Summary:")
            print(f"  Avg Fitness: {avg_fitness:.4f}")
            print(f"  Best Fitness: {self.best_fitness:.4f}")
            print(f"{'*'*60}")

            # Evolution
            selected = self.selection(population, fitness_scores)

            next_population = []
            for i in range(0, len(selected), 2):
                parent1 = selected[i]
                parent2 = selected[i+1] if i+1 < len(selected) else selected[0]

                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                next_population.extend([child1, child2])

            # Elitism
            next_population[0] = self.best_individual.copy()
            population = next_population[:self.population_size]

        return self.best_individual, self.best_fitness


def load_and_prepare_data(filepath=None):
    """Load and prepare diabetes dataset"""
    if filepath:
        # Load custom dataset
        data = pd.read_csv(filepath)
    else:
        # Create sample diabetes dataset (Pima Indians Diabetes style)
        print("Using sample diabetes dataset...")
        np.random.seed(42)
        n_samples = 500

        # Generate synthetic features
        data = pd.DataFrame({
            'Pregnancies': np.random.randint(0, 15, n_samples),
            'Glucose': np.random.normal(120, 30, n_samples),
            'BloodPressure': np.random.normal(70, 12, n_samples),
            'SkinThickness': np.random.normal(29, 10, n_samples),
            'Insulin': np.random.normal(140, 80, n_samples),
            'BMI': np.random.normal(32, 6, n_samples),
            'DiabetesPedigree': np.random.uniform(0.1, 2.5, n_samples),
            'Age': np.random.randint(21, 80, n_samples),
        })

        # Generate outcome based on features (with some logic)
        score = (data['Glucose'] * 0.02 +
                 data['BMI'] * 0.1 +
                 data['Age'] * 0.05 +
                 np.random.normal(0, 2, n_samples))
        data['Outcome'] = (score > 10).astype(int)

    # Separate features and target
    X = data.drop('Outcome', axis=1)
    y = data['Outcome']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    print(f"\nDataset loaded successfully!")
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Features: {X.shape[1]}")
    print(f"Class distribution - Class 0: {sum(y==0)}, Class 1: {sum(y==1)}")

    return X_train, X_val, X_test, y_train, y_val, y_test, scaler


def main():
    """Main execution pipeline"""
    print("\n" + "="*60)
    print("DIABETES PREDICTION WITH NESTED GENETIC ALGORITHM")
    print("="*60)

    # Load data
    X_train, X_val, X_test, y_train, y_val, y_test, scaler = load_and_prepare_data()

    # Step 1: Run Meta-GA to find best GA hyperparameters
    meta_ga = MetaGeneticAlgorithm(
        X_train, y_train, X_val, y_val,
        population_size=6,  # Small for demonstration, increase for real use
        generations=3       # Small for demonstration, increase for real use
    )

    best_ga_params, meta_fitness = meta_ga.evolve()

    print(f"\n{'#'*60}")
    print("FINAL RESULTS - BEST GA HYPERPARAMETERS FOUND:")
    print(f"{'#'*60}")
    for key, value in best_ga_params.items():
        print(f"  {key}: {value}")
    print(f"\nBest validation fitness achieved: {meta_fitness:.4f}")

    # Step 2: Run Inner GA with best GA parameters to find best MLP
    print(f"\n{'='*60}")
    print("FINAL RUN: Training MLP with Optimized Hyperparameters")
    print(f"{'='*60}")

    final_inner_ga = MLPGeneticAlgorithm(
        X_train, y_train, X_val, y_val,
        population_size=best_ga_params['population_size'],
        generations=best_ga_params['generations'],
        mutation_rate=best_ga_params['mutation_rate'],
        crossover_rate=best_ga_params['crossover_rate']
    )

    best_mlp_params, best_fitness = final_inner_ga.evolve()

    # Step 3: Train final model and evaluate on test set
    print(f"\n{'='*60}")
    print("FINAL MODEL EVALUATION ON TEST SET")
    print(f"{'='*60}")

    final_mlp = MLPClassifier(
        hidden_layer_sizes=(
            best_mlp_params['hidden_layer_1'],
            best_mlp_params['hidden_layer_2']
        ),
        learning_rate_init=best_mlp_params['learning_rate_init'],
        alpha=best_mlp_params['alpha'],
        batch_size=best_mlp_params['batch_size'],
        max_iter=300,
        random_state=42
    )

    # Train on full training + validation set
    X_train_full = np.vstack([X_train, X_val])
    y_train_full = np.concatenate([y_train, y_val])

    final_mlp.fit(X_train_full, y_train_full)
    y_pred = final_mlp.predict(X_test)
    y_pred_proba = final_mlp.predict_proba(X_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    auc = roc_auc_score(y_test, y_pred_proba)

    print(f"\nTest Set Performance:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  ROC AUC:  {auc:.4f}")

    print(f"\n{'='*60}")
    print("OPTIMIZATION COMPLETE!")
    print(f"{'='*60}\n")

    return {
        'best_ga_params': best_ga_params,
        'best_mlp_params': best_mlp_params,
        'test_accuracy': accuracy,
        'test_f1': f1,
        'test_auc': auc,
        'model': final_mlp,
        'scaler': scaler
    }


if __name__ == "__main__":
    results = main()


DIABETES PREDICTION WITH NESTED GENETIC ALGORITHM
Using sample diabetes dataset...

Dataset loaded successfully!
Training samples: 320
Validation samples: 80
Test samples: 100
Features: 8
Class distribution - Class 0: 386, Class 1: 114

############################################################
OUTER GA (Meta-GA): Optimizing GA Hyperparameters
Population Size: 6, Generations: 3
############################################################


************************************************************
META-GA Generation 1/3
************************************************************

Evaluating GA Config 1/6:
  Population: 10, Generations: 19
  Mutation: 0.292, Crossover: 0.830

Inner GA: Optimizing MLP Hyperparameters
Population Size: 10, Generations: 19
Mutation Rate: 0.292, Crossover Rate: 0.830
Gen  1 | Avg Fitness: 0.7552 | Best Fitness: 0.7991
Gen  2 | Avg Fitness: 0.7706 | Best Fitness: 0.8055
Gen  3 | Avg Fitness: 0.7714 | Best Fitness: 0.8055
Gen  4 | Avg Fitness: 0.7719 | B